In [1]:
from sklearnex import patch_sklearn

patch_sklearn()

import numpy as np
import scipy.io
import os

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.covariance import LedoitWolf

from pyriemann.estimation import Covariances
from pyriemann.classification import TSclassifier
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.mean import mean_logeuclid
#from pyriemann.classification import SVC

from sklearn.svm import SVC

import h5py

import time

from sklearn.model_selection import StratifiedKFold

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
start_time = time.time()

In [14]:
LoadingPath_Non='.../Ns_100_Individual/Advance'

SavingPath = '.../Ns_100_Individual/Advance/Transformed/Ind_TS_Transformed/Baseline'

In [4]:
Nc = 108

# Functions

In [5]:
import os
import h5py
import numpy as np
import scipy.io

def load_testing_data(testing_loading_path, test_subject_id, n_components=108):
    """
    Load (raw)training and testing datasets for a given subject.
    
    Parameters:
        loading_path (str): Path to the directory containing the dataset files.
        subject_id (int): Subject ID (0-based index).
        n_components (int): Number of components/features to extract.
        
    Returns:
        dict: A dictionary containing all loaded datasets and features.
    """
    # File paths
    testing_dataset_path = os.path.join(testing_loading_path, 'TestingDataset.mat') 
 
    with h5py.File(testing_dataset_path, 'r') as f_testing_dataset:
            testing_dataset = f_testing_dataset['TestingDataset']
            subject_cell_ref = testing_dataset[test_subject_id, 0]
            subject_data = f_testing_dataset[subject_cell_ref]
            X_test_dataset = np.transpose(subject_data['x'][:], (0, 1, 2))
            Y_test_dataset = np.squeeze(subject_data['y'][:])


    # Organize results into a dictionary
    data = {
        "X_test_dataset": X_test_dataset,
        "Y_test_dataset": Y_test_dataset,
    }

    return data


In [6]:
import os
import h5py
import numpy as np
import scipy.io

def load_training_subject_data(training_loading_path, test_subject_id, train_subject_id, n_components=108):
    """
    Load (raw)training and testing datasets for a given subject.
    
    Parameters:
        loading_path (str): Path to the directory containing the dataset files.
        subject_id (int): Subject ID (0-based index).
        n_components (int): Number of components/features to extract.
        
    Returns:
        dict: A dictionary containing all loaded datasets and features.
    """
    # File paths
    training_dataset_path = os.path.join(training_loading_path, 'TrainingDataset.mat') 
 
    with h5py.File(training_dataset_path, 'r') as f_training_dataset:
        training_dataset = f_training_dataset['TrainingDataset']
        training_cell_ref = training_dataset[test_subject_id,0]
        training_cell = f_training_dataset[training_cell_ref]

        # Access the specific train subject inside the nested 1x17 cell
        subject_ref = training_cell[train_subject_id,0]
        subject_data = f_training_dataset[subject_ref]

        X_train_dataset = np.transpose(subject_data['x'][:], (0, 1, 2))
        Y_train_dataset = np.squeeze(subject_data['y'][:])
 

    # Organize results into a dictionary
    data = {
        "X_train_dataset": X_train_dataset,
        "Y_train_dataset": Y_train_dataset,
    }

    return data

In [7]:
# Recentering function 
import numpy as np
from scipy.linalg import fractional_matrix_power
from pyriemann.utils.tangentspace import tangent_space

def Recentering_TSProjection(C, M):
    """
    1. Centers each covariance matrix: M^(-1/2) C M^(-1/2))
    2. Projects to TS (& vectorises) using identity matrix as 'new centre'
    
    Parameters:
        C (ndarray): set of covariance matrices (Nt, Nc, Nc) 
        M (ndarray): Reference matrix (Taken as log-eulid mean)
    
    Returns:
        ndarray: The log-transformed matrix.
    """
    
    Nt, _, _ = C.shape # Number of trials 
    
    M_inv_sqrt = fractional_matrix_power(M, -0.5) # Compute M^(-1/2)
    
    # 1. Recentering step (on each covariance matrix) 
    X_centered = np.array([M_inv_sqrt @ C[i] @ M_inv_sqrt for i in range(Nt)])
    #print(X_centered.shape)
    
    #2. TS projection 
    Centered_M = np.eye(C.shape[1]) # New center after centering is the identity matrix 
    X_TS = tangent_space(X_centered, Centered_M) # Project to TS (& vectorise)

    
    return X_TS

In [8]:
def Rescaling(C):
    """
    Rescale all tangent vectors: c_tilde = c / (1/N_t * sum_n ||c_n||)

    Parameters:
        C (ndarray): Collection of tangent vectors (c_n) to be rescaled

    Returns:
        ndarray: Rescaled tangents vectors `c_tilde`.
    """
    
    Nt = C.shape[0] # Number of vectors
    
    # Check if C contains any 'nan' and replace with 0.0
    if np.isnan(C).any():
        C = np.nan_to_num(C, nan=0.0)
    
    # Compute the average magnitude (1/N_t * sum(||c_n||))
    avg_magnitude = np.sum(np.linalg.norm(C, axis=1)) / Nt

    # Normalize all vectors simultaneously
    c_tilde  = C / avg_magnitude

    return c_tilde 

In [9]:
# Rotation Functions


# Mean per class 
def ClassMean(Dataset, Labels):
    """
    Compute the class-wise mean based on the provided equation:
        s̄_k = (1 / N_k) * Σ s̃_i  (for y_i = k)
    
    Parameters:
        Dataset (ndarray): Tangent vectors, Nt = number of trials, Nf = features per trial.
        Labels (ndarray): The corresponding labels for the dataset, of shape (Nt,).
        
    Returns:
        ndarray: matrix of shape (k, Nf) (k = number of classes)
    """
    unique_classes = np.unique(Labels)  # Find unique classes 
    class_means = []

    for cls in unique_classes: # for each class
        # Select data corresponding to the current class
        class_data = Dataset[Labels == cls]  # Filter rows where label == cls
        N_k = class_data.shape[0]  # Number of trials for class k
        
        # Compute mean
        class_mean = np.mean(class_data, axis=0)
        
        # Store result
        class_means.append(class_mean)
    
    C_bar = np.column_stack(class_means)

    return C_bar

# Load subject's dataset

In [10]:
def load_data(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        return data

In [11]:
TransformedTraining = {}
TransformedTesting = {}
indices_split = {}

subject_range = 'sub001_003'
Fold_Ids = [0, 1, 2]
#Fold_Ids = [0, 1, 2, 3, 4]
#Fold_Ids = [5, 6, 7, 8, 9]
N_train = 17
N_folds = 3

#for test_sub_id in [0]:
for test_sub_id in Fold_Ids:
    print(f"Testing subject {test_sub_id}:")
    test_subject_data = load_testing_data(LoadingPath_Non, test_sub_id, n_components=Nc)


    X_test_dataset = test_subject_data["X_test_dataset"]
    Y_test_dataset = test_subject_data["Y_test_dataset"]
    
    
    # Sanity check
    print("Loaded testing datatsets shape: ")
    print(X_test_dataset.shape)
    
# ---------------------- 1. Recentre (TESTING) - recentres all trials to have centre of mass equal to identity matrix, followed by tangent space mapping (ref = I) ----------------------     
    # Find covariance matrices
    cov_estimator = Covariances(estimator='lwf')
    Test_cov = cov_estimator.transform(X_test_dataset)

    # Find log-euclid centre ('M')
    Test_M = mean_logeuclid(Test_cov)

    # Centre and project to TS (& vectorise)
    Test_Centered = Recentering_TSProjection(Test_cov, Test_M)


    # ---------------------- 1. Recentre (TRAINING) - recentres all trials to have centre of mass equal to identity matrix, followed by tangent space mapping (ref = I) ----------------------     
    Train_Centered_list = []
    Y_train_list=[]
    
    # Load each (ind) tranining dataset
    for train_sub_id in range(N_train):
        #print(f"Training subject {train_sub_id}:")
        train_subject_data = load_training_subject_data(LoadingPath_Non, test_sub_id, train_sub_id, n_components=Nc)


        X_train_dataset = train_subject_data["X_train_dataset"]
        Y_train_sub_dataset = train_subject_data["Y_train_dataset"]
    

        # Sanity check
        # print("Loaded training datatsets shape: ")
        # print(X_train_dataset.shape)
        
        # Find covariance matrices
        cov_estimator = Covariances(estimator='lwf')
        Train_cov = cov_estimator.transform(X_train_dataset)

        # Find log-euclid centre ('M')
        Train_M = mean_logeuclid(Train_cov)

        # Centre and project to TS (& vectorise)
        Train_sub_Centered = Recentering_TSProjection(Train_cov, Train_M)
        
        # Concatenate each subjects' centred vector and corresponding labels 
        Train_Centered_list.append(Train_sub_Centered)  
        Y_train_list.append(Y_train_sub_dataset)
        
    Train_Centered = np.concatenate(Train_Centered_list, axis=0)
    Y_train_dataset = np.concatenate(Y_train_list, axis = 0)
    
    print(f"Labels sizes: Training {Y_train_dataset.shape}, Testing {Y_test_dataset.shape}")
    
    #Sanity Check
    print("Centred shapes: ")
    print(Train_Centered.shape)
    print(Test_Centered.shape)
    
    # ---------------------- 2. Rescale - match matrix dispersion around mean in both 'source' and 'target' (setting the average norm within set to be 1) ----------------------
    Train_Rescale = Rescaling(Train_Centered)
    Test_Rescale = Rescaling(Test_Centered)

    #Sanity Check
    print("Rescaled shapes: ")
    print(Train_Rescale.shape)
    print(Test_Rescale.shape)
    
    # ---------------------- 3. Rotation (Alignment of 'target' vectors) - Align each mean of each class as much as possible (using Eulidea Procrustes procedure)  ----------------------
    # ---------------------- Once target vectors are aligned, can be used with models trained using 'Train_Rescale' ---------------------- 
    # Calculate anchor points for each class 
    
    
    # Split Testing into calibration and testing 
    Test_Rotated_list=[]
    Test_labels_list=[]
    
    # Store indices split per testing subject and fold for consistency across Baseline and SpatFilt
    calibration_idx_list = []
    test_idx_list = []
    
    indices_split[test_sub_id] = {}
    
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42) # 50:50 split calibration and testing
    for fold_idx, (calibration_idx, test_idx) in enumerate(skf.split(Test_Rescale, Y_test_dataset)):
    
        print(f"Fold index: {fold_idx}")
        print(f"Indices for calibration subset: {calibration_idx}")
        print(f"Indices for testing subset: {test_idx}")
        
        # Store per testing subject and fold
        indices_split[test_sub_id][f"Fold{fold_idx + 1}"] = {
            "calibration_idx": calibration_idx,
            "test_idx": test_idx
        }


        X_calibration, X_test = Test_Rescale[calibration_idx], Test_Rescale[test_idx]
        Y_calibration, Y_test = Y_test_dataset[calibration_idx], Y_test_dataset[test_idx]
        


        Train_AnchorPoints=ClassMean(Train_Rescale, Y_train_dataset)
        Test_AnchorPoints=ClassMean(X_calibration, Y_calibration) # Only calibration subset

        # Sanity check
        print("Anchor Points shape: ")
        print(Train_AnchorPoints.shape)
        print(Test_AnchorPoints.shape)

        # Cross-product matrix 
        c_st = Train_AnchorPoints @ Test_AnchorPoints.T

        #Sanity check - should be (Nf, Nf)
        print("c_st shape: ")
        print(c_st.shape) 

        # Perform Singular value decomposition on c_st
        U, D, VT = np.linalg.svd(c_st, full_matrices=False)

        # Find number of Nv vectors that explains 99.9% varaince 
        explained_variance = D**2
        total_variance = np.sum(explained_variance)
        cumulative_explained_variance = np.cumsum(explained_variance) / total_variance
        Nv = np.argmax(cumulative_explained_variance >= 0.999) + 1  # +1 because of 0-based indexing

        U_tilde = U[:, :Nv] # Truncate using only Nv vectors
        VT_tilde = VT[:Nv, :]

        # Sanity check 
        print("truncated U and VT: ")
        print(U_tilde.shape)
        print(VT_tilde.shape)

        print(X_calibration.shape)

        Nt = X_test.shape[0]
        Test_Rotated = np.zeros_like(X_test) # Initialise storage

        # Align each testing trial
        for t in range(Nt):
            Test_Rotated[t] = U_tilde @ VT_tilde @ X_test[t]

        print("Rotated Test: ")
        print(Test_Rotated.shape)

         # Save for this fold
        Test_Rotated_list.append(Test_Rotated)
        Test_labels_list.append(Y_test)

        if test_sub_id not in TransformedTesting:
            TransformedTesting[test_sub_id] = {}

        TransformedTesting[test_sub_id][f"Fold{fold_idx + 1}"] = {
            "x": Test_Rotated,
            "y": Y_test
        }

    if test_sub_id not in TransformedTraining:
        TransformedTraining[test_sub_id] = {}
        
    # Store per subject (transformed training and testing datasets)
    TransformedTraining[test_sub_id] = {
        "x": Train_Rescale,
        "y": Y_train_dataset
    }


Testing subject 0:
Loaded testing datatsets shape: 
(2777, 108, 100)
Labels sizes: Training (43450,), Testing (2777,)
Centred shapes: 
(43450, 5886)
(2777, 5886)
Rescaled shapes: 
(43450, 5886)
(2777, 5886)
Fold index: 0
Indices for calibration subset: [   1    2    6 ... 2771 2773 2776]
Indices for testing subset: [   0    3    4 ... 2772 2774 2775]
Anchor Points shape: 
(5886, 2)
(5886, 2)
c_st shape: 
(5886, 5886)
truncated U and VT: 
(5886, 2)
(2, 5886)
(1388, 5886)
Rotated Test: 
(1389, 5886)
Fold index: 1
Indices for calibration subset: [   0    3    4 ... 2772 2774 2775]
Indices for testing subset: [   1    2    6 ... 2771 2773 2776]
Anchor Points shape: 
(5886, 2)
(5886, 2)
c_st shape: 
(5886, 5886)
truncated U and VT: 
(5886, 2)
(2, 5886)
(1389, 5886)
Rotated Test: 
(1388, 5886)
Testing subject 1:
Loaded testing datatsets shape: 
(2808, 108, 100)
Labels sizes: Training (43419,), Testing (2808,)
Centred shapes: 
(43419, 5886)
(2808, 5886)
Rescaled shapes: 
(43419, 5886)
(2808, 

In [12]:
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time/60, "minutes")

Elapsed time: 20.86174545288086 minutes


In [15]:
SavingPath

'.../Ns_100_Individual/Advance/Transformed/Ind_TS_Transformed/Baseline'

In [16]:
import os
import pickle

# Define saving paths for the variables
training_saving_path = os.path.join(SavingPath, f'TransformedTraining_{subject_range}.pkl')
testing_saving_path = os.path.join(SavingPath, f'TransformedTesting_{subject_range}.pkl')

indices_saving_path = os.path.join(SavingPath, f'Indices_Split_{subject_range}.pkl')

# training_saving_path = os.path.join(SavingPath, 'TransformedTraining.pkl')
# testing_saving_path = os.path.join(SavingPath, 'TransformedTesting.pkl')

# indices_saving_path = os.path.join(SavingPath, 'Indices_Split.pkl')


# Save TransformedTraining
with open(training_saving_path, 'wb') as file:
    pickle.dump(TransformedTraining, file)

# Save TransformedTesting
with open(testing_saving_path, 'wb') as file:
    pickle.dump(TransformedTesting, file)

# Save Indices
with open(indices_saving_path, 'wb') as file:
    pickle.dump(indices_split, file)

# Extra/Old scripts